In [8]:
import pandas as pd
from scipy.stats import chi2_contingency, f_oneway

# 데이터 로드 및 전처리
data = pd.read_csv('../datasets/customer-segmentation_Train.csv')

# 결측값 처리
data_cleaned = data.copy()
data_cleaned['Ever_Married'].fillna('Unknown', inplace=True)
data_cleaned['Profession'].fillna('Unknown', inplace=True)
data_cleaned['Work_Experience'].fillna(data_cleaned['Work_Experience'].median(), inplace=True)
data_cleaned['Family_Size'].fillna(data_cleaned['Family_Size'].median(), inplace=True)


In [9]:
# 데이터 요약
print("데이터 요약:")
print(data_cleaned.info())

# 각 컬럼별 고유값 확인
print("\n각 컬럼별 고유값:")
for col in data_cleaned.columns:
    print(f"{col}: {data_cleaned[col].unique()}")

# 결측값 개수 확인
print("\n결측값 개수:")
print(data_cleaned.isnull().sum())

# 기본 통계량 확인
print("\n데이터 기본 통계량:")
print(data_cleaned.describe())

# 특정 범주형 변수의 빈도수 확인 (예: 성별)
print("\n성별 빈도수:")
print(data_cleaned['Gender'].value_counts())

데이터 요약:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     8068 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       8068 non-null   object 
 6   Work_Experience  8068 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      8068 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB
None

각 컬럼별 고유값:
ID: [462809 462643 466315 ... 465406 467299 461879]
Gender: ['Male' 'Female']
Ever_Married: ['No' 'Yes' 'Unknown']
Age: [22 38 67 40 56 32 33 61 55 26 19 70 58 41 31 79 49 18 36 35 45 42 83 27
 28 47 29 57 

In [10]:
# 결과 저장
results = {}

# 1. 성별에 따른 지출 점수 분석 (Chi-Square Test)
# Crosstab 생성 및 확인
gender_spending_table = pd.crosstab(data_cleaned['Gender'], data_cleaned['Spending_Score'])
print("\n성별과 지출 점수 간 교차표:")
print(gender_spending_table)

# 데이터 수 확인
print("\n데이터 수 확인:")
print(f"성별 갯수: {data_cleaned['Gender'].nunique()}")
print(f"지출 점수 갯수: {data_cleaned['Spending_Score'].nunique()}")

chi2, p, _, _ = chi2_contingency(gender_spending_table)
results['Gender_vs_Spending_Score'] = {'p-value': p, 'Chi2': chi2}


성별과 지출 점수 간 교차표:
Spending_Score  Average  High   Low
Gender                             
Female              831   490  2330
Male               1143   726  2548

데이터 수 확인:
성별 갯수: 2
지출 점수 갯수: 3


In [11]:
# 2. 결혼 여부에 따른 직업 분포 분석 (Chi-Square Test)
# Crosstab 생성 및 확인
married_profession_table = pd.crosstab(data_cleaned['Ever_Married'], data_cleaned['Profession'])
print("\n결혼 여부와 직업 분포 간 교차표:")
print(married_profession_table)

# 데이터 수 확인
print("\n데이터 수 확인:")
print(f"결혼 여부 고유값: {data_cleaned['Ever_Married'].nunique()}")
print(f"직업 고유값: {data_cleaned['Profession'].nunique()}")

chi2, p, _, _ = chi2_contingency(married_profession_table)
results['Ever_Married_vs_Profession'] = {'p-value': p, 'Chi2': chi2}


결혼 여부와 직업 분포 간 교차표:
Profession    Artist  Doctor  Engineer  Entertainment  Executive  Healthcare  \
Ever_Married                                                                   
No               713     366       267            358         37        1153   
Unknown           29      11        17             12         12          34   
Yes             1774     311       415            579        550         145   

Profession    Homemaker  Lawyer  Marketing  Unknown  
Ever_Married                                         
No                  112      40        189       50  
Unknown               6       8          7        4  
Yes                 128     575         96       70  

데이터 수 확인:
결혼 여부 고유값: 3
직업 고유값: 10


In [12]:
# 3. 나이에 따른 지출 점수 분석 (ANOVA)
# 나이와 지출 점수의 상관관계 확인
correlation_age_spending = data_cleaned['Age'].corr(data_cleaned['Spending_Score'])
print("\n나이와 지출 점수 상관관계:")
print(f"Correlation: {correlation_age_spending}")

# 지출 점수별 나이 데이터 수 확인
print("\n지출 점수별 나이 데이터 분포:")
print(data_cleaned.groupby('Spending_Score')['Age'].count())
spending_groups = [data_cleaned['Age'][data_cleaned['Spending_Score'] == category]
                   for category in data_cleaned['Spending_Score'].unique()]
anova_stat, anova_p = f_oneway(*spending_groups)
results['Age_vs_Spending_Score'] = {'p-value': anova_p, 'F-stat': anova_stat}

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [13]:
# 4. 직업에 따른 근무 연수 분석 (ANOVA)
# 직업별 근무 연수 데이터 요약
print("\n직업별 근무 연수 요약:")
print(data_cleaned.groupby('Profession')['Work_Experience'].describe())

# 직업과 근무 연수의 상관관계 확인
correlation_work_experience = data_cleaned['Work_Experience'].corr(data_cleaned['Age'])
print("\n나이와 근무 연수 간 상관관계:")
print(f"Correlation: {correlation_work_experience}")

experience_groups = [data_cleaned['Work_Experience'][data_cleaned['Profession'] == profession]
                     for profession in data_cleaned['Profession'].unique()]
anova_stat, anova_p = f_oneway(*experience_groups)
results['Profession_vs_Work_Experience'] = {'p-value': anova_p, 'F-stat': anova_stat}



직업별 근무 연수 요약:
                count      mean       std  min  25%  50%  75%   max
Profession                                                         
Artist         2516.0  2.565183  3.277389  0.0  0.0  1.0  4.0  14.0
Doctor          688.0  2.485465  3.391885  0.0  0.0  1.0  4.0  14.0
Engineer        699.0  2.433476  3.204504  0.0  0.0  1.0  4.0  14.0
Entertainment   949.0  2.594310  3.318979  0.0  0.0  1.0  4.0  14.0
Executive       599.0  2.170284  2.998503  0.0  0.0  1.0  2.0  13.0
Healthcare     1332.0  2.415165  3.153643  0.0  0.0  1.0  3.0  14.0
Homemaker       246.0  5.686992  4.216052  0.0  1.0  8.0  9.0  14.0
Lawyer          623.0  1.199037  2.055234  0.0  0.0  1.0  1.0  14.0
Marketing       292.0  2.400685  3.297463  0.0  0.0  1.0  3.0  14.0
Unknown         124.0  2.104839  2.926799  0.0  0.0  1.0  2.0  12.0

나이와 근무 연수 간 상관관계:
Correlation: -0.17734425014106608


In [16]:
# 가족 수와 지출 점수 간 상관관계 확인
# Family_Size와 Spending_Score 데이터 타입 확인 및 변환
print(data_cleaned[['Family_Size', 'Spending_Score']].dtypes)

# Family_Size와 Spending_Score의 값 확인
print("\nFamily_Size 고유값:")
print(data_cleaned['Family_Size'].unique())
print("\nSpending_Score 고유값:")
print(data_cleaned['Spending_Score'].unique())

# 숫자로 변환 (필요 시 에러 값은 NaN으로 처리)
data_cleaned['Family_Size'] = pd.to_numeric(data_cleaned['Family_Size'], errors='coerce')
data_cleaned['Spending_Score'] = pd.to_numeric(data_cleaned['Spending_Score'], errors='coerce')

# 변환 후 결측값 처리 (필요 시)
data_cleaned['Family_Size'].fillna(data_cleaned['Family_Size'].median(), inplace=True)
data_cleaned['Spending_Score'].fillna(data_cleaned['Spending_Score'].median(), inplace=True)

# 상관계수 계산
correlation_family_spending = data_cleaned['Family_Size'].corr(data_cleaned['Spending_Score'])
print("\n가족 수와 지출 점수 간 상관관계:")
print(f"Correlation: {correlation_family_spending}")

# 지출 점수별 가족 수 데이터 분포
print("\n지출 점수별 가족 수 데이터 분포:")
print(data_cleaned.groupby('Spending_Score')['Family_Size'].count())

family_size_groups = [
    data_cleaned['Family_Size'][data_cleaned['Spending_Score'] == category].dropna()
    for category in data_cleaned['Spending_Score'].unique()
]

# 빈 그룹 제거
family_size_groups = [group for group in family_size_groups if len(group) > 0]

# 그룹 확인
print(f"\n유효한 그룹 수: {len(family_size_groups)}")
for i, group in enumerate(family_size_groups):
    print(f"그룹 {i + 1}: {len(group)}개의 데이터")

# ANOVA 실행
if len(family_size_groups) > 1:
    anova_stat, anova_p = f_oneway(*family_size_groups)
    results['Family_Size_vs_Spending_Score'] = {'p-value': anova_p, 'F-stat': anova_stat}
    print("\nANOVA 결과:")
    print(f"F-stat: {anova_stat}, p-value: {anova_p}")
else:
    print("\nANOVA를 실행하기에 유효한 그룹이 충분하지 않습니다.")

# 결과 출력
for test, result in results.items():
    print(f"{test}: {result}")

Family_Size       float64
Spending_Score    float64
dtype: object

Family_Size 고유값:
[4. 3. 1. 2. 6. 5. 8. 7. 9.]

Spending_Score 고유값:
[nan]

가족 수와 지출 점수 간 상관관계:
Correlation: nan

지출 점수별 가족 수 데이터 분포:
Series([], Name: Family_Size, dtype: int64)

유효한 그룹 수: 0

ANOVA를 실행하기에 유효한 그룹이 충분하지 않습니다.
Gender_vs_Spending_Score: {'p-value': 9.102926059917657e-08, 'Chi2': 32.42416967413639}
Ever_Married_vs_Profession: {'p-value': 0.0, 'Chi2': 2110.4447402437395}
Profession_vs_Work_Experience: {'p-value': 3.457356160939731e-70, 'F-stat': 39.86638039452044}


/usr/local/lib/python3.8/dist-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
